In [1]:
import numpy as np
import pandas as pd
import re
import string
import nltk
import torch

from unidecode import unidecode
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from transformers import AutoModel, AutoTokenizer

In [3]:
stopwords = list(set([unidecode(w) for w in nltk.corpus.stopwords.words("portuguese")]))
stopwords.extend(list(set([unidecode(w) for w in nltk.corpus.stopwords.words("english")])))
stopwords.extend(["rt", "@user"])

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")
model = AutoModel.from_pretrained("xlm-roberta-large").to("cuda")

In [4]:
df = pd.read_csv('datasets/train_expanded.csv')
df 

In [13]:
def clean_text(dataframe, col):
    dataframe[col] = dataframe[col].apply(lambda x: re.sub("#[^ ]+", "", x))
    dataframe[col] = dataframe[col].apply(lambda x: re.sub(r"\d+", "", x))
    dataframe[col] = dataframe[col].apply(lambda x: re.sub("http[^ ]+", "", x))
    dataframe[col] = dataframe[col].apply(lambda x: x.translate(str.maketrans("", "", string.punctuation)))
    dataframe[col] = dataframe[col].apply(lambda x: x.lower().strip())
    return dataframe[col]

def tokenize(dataframe, col):
    dataframe[col] = dataframe[col].apply(lambda x: tokenizer(x,padding=True,truncation=True,max_length=512,return_tensors="pt",))
    return dataframe[col]


In [15]:
df['norm_text_clean_v2'] = clean_text(df, 'norm_text_clean')
df['norm_text_clean_v2'] = tokenize(df, 'norm_text_clean_v2')
df = df[['norm_text_clean_v2','label']]

KeyError: 'norm_text_clean'

AttributeError: 'DataFrame' object has no attribute 'map'

In [ ]:
dataset_tokenized = dataset.map(self._tokenize, batched=True, batch_size=None)
        try:
            dataset_tokenized.set_format(
                type="torch", columns=["input_ids", "attention_mask", "label"]
            )
        except ValueError:
            dataset_tokenized.set_format(
                type="torch", columns=["input_ids", "attention_mask"]
            )

        dataset_hidden = dataset_tokenized.map(
            self._extract_hidden_states, batched=True
        )

In [ ]:
vector = CountVectorizer()
X_vector = vector.fit_transform(X)
X_train, x, Y_train, y = train_test_split(X_vector, Y, test_size=0.2, random_state=42)

In [ ]:
model = LogisticRegression(C=1, max_iter=300, solver='saga')
model.fit(X_train, Y_train)

In [ ]:
y_pred_best = model.predict(x)
accuracy_best = accuracy_score(y, y_pred_best)
print("Acuracia:", accuracy_best)
print(classification_report(y, y_pred_best))